In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN' # your Foursquare ID
CLIENT_SECRET = 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN
CLIENT_SECRET:GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE


In [3]:
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent foursquare_agent, as shown below.
#**** Finding the longitude and latitude by address input (as shown in Foursquare) ****
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [4]:
#search italian dish serving restaurants near the ll=40.7149555 and -74.0153365
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format('MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 40.7149555, -74.0153365, 20180604, 'search', 500, 30)
url

'https://api.foursquare.com/v2/venues/search?client_id=MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN&client_secret=GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE&ll=40.7149555,-74.0153365&v=20180604&query=search&radius=500&limit=30'

In [9]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e0642083907e72fab563d31'},
 'response': {'venues': [{'id': '4e7e41b87beb96a21aba4704',
    'name': 'Coffee &  Donut Cart',
    'location': {'address': 'Barclay st',
     'crossStreet': 'Church st',
     'lat': 40.71268922492194,
     'lng': -74.00998547673225,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71268922492194,
       'lng': -74.00998547673225}],
     'distance': 517,
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Barclay st (Church st)',
      'New York, NY',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1cb941735',
      'name': 'Food Truck',
      'pluralName': 'Food Trucks',
      'shortName': 'Food Truck',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1577468482',
    'hasPerk': False}]}}

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4e7e41b87beb96a21aba4704,Barclay st,US,New York,United States,Church st,517,"[Barclay st (Church st), New York, NY, United ...","[{'label': 'display', 'lat': 40.71268922492194...",40.712689,-74.009985,NY,Coffee & Donut Cart,v-1577468482


In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,state,id
0,Coffee & Donut Cart,Food Truck,Barclay st,US,New York,United States,Church st,517,"[Barclay st (Church st), New York, NY, United ...","[{'label': 'display', 'lat': 40.71268922492194...",40.712689,-74.009985,NY,4e7e41b87beb96a21aba4704


In [12]:
#visualizing the results
dataframe_filtered.name

0    Coffee &  Donut Cart
Name: name, dtype: object

In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### Explore a Given Venue

In [24]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 20180604)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN&client_secret=GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE&v=20180604'

In [26]:
#Let's explore the closest Italian restaurant -- Harry's Italian Pizza Bar
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restauran

In [27]:
#Send GET request for result

result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restauran

In [28]:
#Get the venue's overall rating

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [29]:
#That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 20180604)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [30]:
#Since this restaurant has no ratings, let's check the third restaurant.

venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 20180604)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


In [31]:
#Get the number of tips

result['response']['venue']['tips']['count']

17

In [34]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 20180604, 30)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e06469b949393001d220799'},
 'response': {'tips': {'count': 17,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'lastVoteText': 'Upvoted 1 week ago',
     'lastUpvoteTimestamp': 1576365433,
     'todo': {'count': 0},
     'user': {'id': '0',
      'firstName': 'Nick',
      'lastName': 'E',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/blank_boy.png'}},
     'authorInteractionType': 'liked'}]}}}

In [35]:
###Get tips and list of associated features

tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

In [36]:
#Format column width and display all tips

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,E,male,0


In [40]:
#display features associated with user
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format('484542633', 'MJIQTZRMZ0CPSO3WLP1GKGLOCH0VL0QQIRYNJ4SUJRIL1ZLN', 'GPTQ0ASLU1VNQSI5QZ0MRWUOLSSDAFKLXLGE3LIM144DUKXE', 20180604) 
# define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

KeyError: 'user'